In [ ]:
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import muax as mm
from src.envs import HeatEnv
from src.fem import Topology
from const import *
from util import *
import numpy as np
import scipy.sparse as ss
from muax.test import test
import gymnasium as gym
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import haiku as hk
import src.HeatNN as nn

In [ ]:
def adjm(n, m):
    num_nodes = n * m
    A = np.zeros((num_nodes, num_nodes), dtype=int)
    for i in range(num_nodes):
        r = i // m
        c = i % m
        if r > 0:
            A[i, i - m] = 1  
        if r < n - 1:
            A[i, i + m] = 1  
        if c > 0:
            A[i, i - 1] = 1  
        if c < m - 1:
            A[i, i + 1] = 1
    #for i in range(num_nodes):
    #    for j in range(num_nodes):
    #        if i%m == m-1 and j%m == m-1:
    #            A[i, j] = 0
    #            A[j, i] = 0
    A[0, 1] = 0
    A[1, 0] = 0
    A[2, 3] = 0
    A[3, 2] = 0
    A[-3, -4] = 0
    A[-4, -3] = 0
    A[-1, -2] = 0
    A[-2, -1] = 0
    return A

def incm(adjacency_matrix):
    num_nodes = adjacency_matrix.shape[0]
    edges = []
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if adjacency_matrix[i, j] == 1:
                edges.append((i, j))
    num_edges = len(edges)
    incidence_matrix = np.zeros((num_nodes, num_edges), dtype=int)
    for edge_index, (start_node, end_node) in enumerate(edges):
        incidence_matrix[start_node, edge_index] = 1
        incidence_matrix[end_node, edge_index] = -1
    return incidence_matrix

def crtaj(d):
    fig = plt.figure()
    ax = fig.add_subplot()
    norm = Normalize(vmin=d.C.min(), vmax=d.C.max())
    cmap = plt.get_cmap('coolwarm')
    ax.scatter(d.coord[:, 0], d.coord[:, 1], s=5)
    for i in range(d.size):
        ax.plot(d.coord[d.G[:, [i]].indices, 0], d.coord[d.G[:, [i]].indices, 1], color=cmap(norm(d.C[i])))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([]) 
    cbar = plt.colorbar(sm)
    cbar.set_label('C', rotation=90)

In [ ]:
dima, dimb = 4, 4
A = incm(adjm(dima, dimb))
coord = np.vstack(i.flatten() for i in np.meshgrid(np.linspace(1, 10, dima), np.linspace(1, 10, dimb))).T

@jax.jit
def fun(x):
    return jnp.sqrt(jnp.sum(x**2))

cond = 0.01*np.ones((A.shape[1],))